In [ ]:
import os
import json
import subprocess
import geopandas as gpd
from shapely.geometry import box
from shapely.ops import unary_union
import laspy
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

start_time = time.time()  # ⏱️ Début

# === Paramètres ===
buffer_distance = 50
max_workers = 8  # ← Ajuste selon ton CPU


copc_folder = "Dalles_laz"
parcel_file = "parcelles_filtrées.geojson"
output_folder = "Dalles_laz_perim50"
pdal_path = r"C:\Users\victor.nowak\AppData\Local\anaconda3\envs\pdalpy\Library\bin\pdal.exe"

# === Charger les parcelles
parcelles = gpd.read_file(parcel_file).to_crs(epsg=2154)
buffered_union = unary_union(parcelles.buffer(buffer_distance))
polygon_json = json.dumps(gpd.GeoSeries([buffered_union]).__geo_interface__["features"][0]["geometry"])

# === Étendue des fichiers
def get_laz_extent(file_path):
    try:
        with laspy.open(file_path) as f:
            h = f.header
            return box(h.mins[0], h.mins[1], h.maxs[0], h.maxs[1])
    except Exception as e:
        print(f"Erreur lecture {file_path}: {e}")
        return None

# === Fonction de traitement
def process_file(index, file_path):
    extent = get_laz_extent(file_path)
    if extent and extent.intersects(buffered_union):
        out_path = os.path.join(output_folder, f"crop_{index:04d}.laz")
        pipeline = [
            {"type": "readers.copc", "filename": file_path},
            {"type": "filters.crop", "polygon": polygon_json},
            {"type": "writers.las", "filename": out_path, "compression": "laszip"}
        ]
        pipeline_file = f"pipeline_{index:04d}.json"
        with open(pipeline_file, "w") as f:
            json.dump(pipeline, f)
        result = subprocess.run([pdal_path, "pipeline", pipeline_file], capture_output=True, text=True)
        os.remove(pipeline_file)
        if result.returncode == 0:
            return f"✅ {os.path.basename(file_path)} traité"
        else:
            return f"❌ {os.path.basename(file_path)} échoué\n{result.stderr}"
    else:
        return f"⏭️ {os.path.basename(file_path)} ignoré (pas d'intersection)"

# === Lancer les traitements en parallèle
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

copc_files = [os.path.join(copc_folder, f) for f in os.listdir(copc_folder) if f.endswith(".copc.laz")]
tasks = []

print(f"🚀 Traitement parallèle de {len(copc_files)} fichiers...")

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(process_file, i, path) for i, path in enumerate(copc_files)]
    for future in as_completed(futures):
        print(future.result())
end_time = time.time()  # ⏱️ Fin
duration = round(end_time - start_time, 2)

print(f"🕒 Temps d'exécution : {duration} secondes")